In [85]:
import pandas as pd 
import json

df1 = pd.read_csv('mturk_results/Batch_5320889_batch_results_first10_pilot.csv', encoding='utf-8')
df2 = pd.read_csv('mturk_results/Batch_5321077_batch_results_next40.csv', encoding='utf-8')
df3 = pd.read_csv('mturk_results/Batch_5321100_batch_results_last50.csv', encoding='utf-8')

# merge the two dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)
df.columns

Index(['HITId', 'HITTypeId', 'Title', 'Description', 'Keywords', 'Reward',
       'CreationTime', 'MaxAssignments', 'RequesterAnnotation',
       'AssignmentDurationInSeconds', 'AutoApprovalDelayInSeconds',
       'Expiration', 'NumberOfSimilarHITs', 'LifetimeInSeconds',
       'AssignmentId', 'WorkerId', 'AssignmentStatus', 'AcceptTime',
       'SubmitTime', 'AutoApprovalTime', 'ApprovalTime', 'RejectionTime',
       'RequesterFeedback', 'WorkTimeInSeconds', 'LifetimeApprovalRate',
       'Last30DaysApprovalRate', 'Last7DaysApprovalRate', 'Input.q_strings',
       'Answer.taskAnswers', 'Approve', 'Reject'],
      dtype='object')

In [86]:
submission_df = df[['WorkerId','WorkTimeInSeconds','Answer.taskAnswers']]
submission_df

,WorkerId,WorkTimeInSeconds,Answer.taskAnswers
0,A1WTHBAWHRVU4P,559,"[{""104_B_ours_instag_q1"":{""instag"":false,""ours..."
1,A1G1O82VQI14Y1,189,"[{""074_A_ga_ours_q1"":{""ga"":false,""ours"":true},..."
2,A2DXOYMV4TJWLF,377,"[{""074_B_ours_instag_q1"":{""instag"":false,""ours..."
3,A1IRR3HIAYF363,452,"[{""218_B_ours_instag_q1"":{""instag"":false,""ours..."
4,AXPTF739HYEZX,517,"[{""074_B_talkg_ours_q1"":{""ours"":true,""talkg"":f..."
...,...,...,...
115,ABL6IBQNPU59Z,706,"[{""253_B_instag_ours_q1"":{""instag"":true,""ours""..."
116,APLYJOP26WC4W,491,"[{""104_A_instag_ours_q1"":{""instag"":true,""ours""..."
117,A5OH0SQJYFO2H,1089,"[{""074_A_instag_ours_q1"":{""instag"":true,""ours""..."
118,A9T4KLSX4JUUP,298,"[{""253_A_ga_ours_q1"":{""ga"":false,""ours"":true},..."


In [87]:
# answer stats per worker
# recall: each worker answers 9 x 3 = 27 questions, full score is 27
for i in range(len(submission_df)):
    print(f"workerid: {submission_df.at[i, 'WorkerId']}, worktime: {submission_df.at[i, 'WorkTimeInSeconds']}")
    answer_json = json.loads(submission_df.at[i, 'Answer.taskAnswers'])
    methods = ['ours', 'ga', 'talkg', 'instag']
    score_dict = {method: 0 for method in methods}
    for question, answer in answer_json[0].items():
        # print(f"question: {question}, answer: {answer}") # see individual answers
        for method, score in answer.items():
            if method in methods:
                score_dict[method] += int(score)
    print(f"score_dict: {score_dict}")




workerid: A1WTHBAWHRVU4P, worktime: 559
score_dict: {'ours': 24, 'ga': 0, 'talkg': 3, 'instag': 0}
workerid: A1G1O82VQI14Y1, worktime: 189
score_dict: {'ours': 8, 'ga': 7, 'talkg': 7, 'instag': 5}
workerid: A2DXOYMV4TJWLF, worktime: 377
score_dict: {'ours': 10, 'ga': 4, 'talkg': 7, 'instag': 6}
workerid: A1IRR3HIAYF363, worktime: 452
score_dict: {'ours': 25, 'ga': 1, 'talkg': 1, 'instag': 0}
workerid: AXPTF739HYEZX, worktime: 517
score_dict: {'ours': 21, 'ga': 4, 'talkg': 2, 'instag': 0}
workerid: A2T8AISA63GR31, worktime: 553
score_dict: {'ours': 24, 'ga': 0, 'talkg': 3, 'instag': 0}
workerid: A25GP17YDFVXPC, worktime: 198
score_dict: {'ours': 18, 'ga': 3, 'talkg': 4, 'instag': 2}
workerid: AIOJAGGDX2E2Z, worktime: 488
score_dict: {'ours': 20, 'ga': 0, 'talkg': 4, 'instag': 3}
workerid: A1S1HDM5HBGT1B, worktime: 405
score_dict: {'ours': 15, 'ga': 3, 'talkg': 3, 'instag': 6}
workerid: A2NC3DH5SD5OPJ, worktime: 554
score_dict: {'ours': 17, 'ga': 3, 'talkg': 3, 'instag': 4}
workerid: A1P

In [104]:
# method score distribution by subj+A/B

# create df (workerid, worktime, subj, speech, methodA, methodB, methodA_wins, methodB_wins, ours_wins, ga_wins, talkg_wins, instag_wins)
df_list = []
for i in range(len(submission_df)):
    workerId = submission_df.at[i, 'WorkerId']
    worktime = submission_df.at[i, 'WorkTimeInSeconds']

    answer_json = json.loads(submission_df.at[i, 'Answer.taskAnswers'])
    methods = ['ours', 'ga', 'talkg', 'instag']
    # score_dict = {method: 0 for method in methods}
    for question, answer in answer_json[0].items():
        subj = question.split('_')[0]
        speech = question.split('_')[1]
        methodA = question.split('_')[2]
        methodB = question.split('_')[3]
        question_id = question.split('_')[4]

        methodA_wins = answer[methodA]
        methodB_wins = answer[methodB]

        # using wins, get 0,1,0,0 for ours, ga, talkg, instag
        ours_wins = 0
        ga_wins = 0
        talkg_wins = 0
        instag_wins = 0
        if methodA == 'ours':
            ours_wins = int(methodA_wins)
        elif methodA == 'ga':
            ga_wins = int(methodA_wins)
        elif methodA == 'talkg':
            talkg_wins = int(methodA_wins)
        elif methodA == 'instag':
            instag_wins = int(methodA_wins)
        
        if methodB == 'ours':
            ours_wins = int(methodB_wins)
        elif methodB == 'ga':
            ga_wins = int(methodB_wins)
        elif methodB == 'talkg':
            talkg_wins = int(methodB_wins)
        elif methodB == 'instag':
            instag_wins = int(methodB_wins)

        # create tuple
        df_list.append((workerId, worktime, subj, speech, question_id, methodA, methodB, methodA_wins, methodB_wins, ours_wins, ga_wins, talkg_wins, instag_wins))

# create df
df = pd.DataFrame(df_list, columns=['WorkerId', 'WorkTimeInSeconds', 'subj', 'speech', 'q_id', 'methodA', 'methodB', 'methodA_wins', 'methodB_wins', 'ours_wins', 'ga_wins', 'talkg_wins', 'instag_wins'])
df

,WorkerId,WorkTimeInSeconds,subj,speech,q_id,methodA,methodB,methodA_wins,methodB_wins,ours_wins,ga_wins,talkg_wins,instag_wins
0,A1WTHBAWHRVU4P,559,104,B,q1,ours,instag,True,False,1,0,0,0
1,A1WTHBAWHRVU4P,559,104,B,q2,ours,instag,True,False,1,0,0,0
2,A1WTHBAWHRVU4P,559,104,B,q3,ours,instag,True,False,1,0,0,0
3,A1WTHBAWHRVU4P,559,218,A,q1,ours,ga,True,False,1,0,0,0
4,A1WTHBAWHRVU4P,559,218,A,q2,ours,ga,True,False,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,A2KWD81ZH7T234,977,306,A,q2,ours,talkg,True,False,1,0,0,0
3236,A2KWD81ZH7T234,977,306,A,q3,ours,talkg,True,False,1,0,0,0
3237,A2KWD81ZH7T234,977,306,B,q1,talkg,ours,False,True,1,0,0,0
3238,A2KWD81ZH7T234,977,306,B,q2,talkg,ours,False,True,1,0,0,0


In [89]:
df_binary = df.drop(columns=['methodA_wins', 'methodB_wins', 'methodA', 'methodB'])
df_binary

,WorkerId,WorkTimeInSeconds,subj,speech,q_id,ours_wins,ga_wins,talkg_wins,instag_wins
0,A1WTHBAWHRVU4P,559,104,B,q1,1,0,0,0
1,A1WTHBAWHRVU4P,559,104,B,q2,1,0,0,0
2,A1WTHBAWHRVU4P,559,104,B,q3,1,0,0,0
3,A1WTHBAWHRVU4P,559,218,A,q1,1,0,0,0
4,A1WTHBAWHRVU4P,559,218,A,q2,1,0,0,0
...,...,...,...,...,...,...,...,...,...
3235,A2KWD81ZH7T234,977,306,A,q2,1,0,0,0
3236,A2KWD81ZH7T234,977,306,A,q3,1,0,0,0
3237,A2KWD81ZH7T234,977,306,B,q1,1,0,0,0
3238,A2KWD81ZH7T234,977,306,B,q2,1,0,0,0


In [90]:
df_binary['WorkTimeInSeconds'].describe()

count    3240.000000
mean      802.366667
std       388.707773
min       176.000000
25%       503.000000
50%       706.000000
75%      1013.000000
max      1731.000000
Name: WorkTimeInSeconds, dtype: float64

In [91]:
df_binary.groupby(['WorkerId','WorkTimeInSeconds']).sum(['ours_wins', 'ga_wins', 'talkg_wins', 'instag_wins']).reset_index()

,WorkerId,WorkTimeInSeconds,ours_wins,ga_wins,talkg_wins,instag_wins
0,A105YRJ9M16N2P,552,9,6,9,3
1,A12HYC1E6I9Q7B,793,24,0,3,0
2,A12ZAHBWT004A0,1258,18,0,3,6
3,A13K6S6AZNO4U0,891,20,0,6,1
4,A15E6XNFAQ2WMU,752,13,4,5,5
...,...,...,...,...,...,...
115,AWY8ZMGJ7RO9Q,928,14,3,6,4
116,AX8D3L76H0JR7,289,11,6,6,4
117,AXDD3EC2J3LI9,395,18,0,6,3
118,AXPTF739HYEZX,517,21,4,2,0


In [92]:
# Q: what subject+AB did ours lose the most?
df_binary.drop(columns=['WorkTimeInSeconds']).groupby(['subj','speech']).sum(['ours_wins', 'ga_wins', 'talkg_wins', 'instag_wins']).reset_index()

,subj,speech,ours_wins,ga_wins,talkg_wins,instag_wins
0,074,A,129,14,15,22
1,074,B,149,8,8,15
2,104,A,112,20,26,22
3,104,B,120,14,23,23
4,218,A,119,13,28,20
5,218,B,136,3,22,19
6,253,A,137,16,12,15
7,253,B,110,11,38,21
8,264,A,94,23,34,29
9,264,B,126,13,23,18


In [93]:
# Q: what question (1,2,3) did ours lose the most?
df_binary.drop(columns=['WorkTimeInSeconds']).groupby(['q_id']).sum(['ours_wins', 'ga_wins', 'talkg_wins', 'instag_wins']).reset_index()

# finding: we win in realness, lip sync & quality debatable

,q_id,ours_wins,ga_wins,talkg_wins,instag_wins
0,q1,717,94,135,134
1,q2,728,87,147,118
2,q3,696,87,164,133


In [102]:
all_wins = df_binary.drop(columns=['WorkTimeInSeconds']).groupby(['q_id']).sum(['ours_wins', 'ga_wins', 'talkg_wins', 'instag_wins']).reset_index()

all_wins = all_wins.drop(columns=['ours_wins'])
# divide all numbers in all_wins by 360
all_wins['ours_wins_ga'] = 1- all_wins['ga_wins'] / 360
all_wins['ours_wins_talkg'] = 1 - all_wins['talkg_wins'] / 360
all_wins['ours_wins_instag'] = 1 - all_wins['instag_wins'] / 360

all_wins

,q_id,ga_wins,talkg_wins,instag_wins,ours_wins_ga,ours_wins_talkg,ours_wins_instag
0,q1,94,135,134,0.738889,0.625000,0.627778
1,q2,87,147,118,0.758333,0.591667,0.672222
2,q3,87,164,133,0.758333,0.544444,0.630556


In [ ]:
all_wins

In [94]:
# Q: what subject+AB+q did ours lose the most?
df_binary.drop(columns=['WorkTimeInSeconds']).groupby(['subj','speech','q_id']).sum(['ours_wins', 'ga_wins', 'talkg_wins', 'instag_wins']).reset_index()

,subj,speech,q_id,ours_wins,ga_wins,talkg_wins,instag_wins
0,074,A,q1,41,6,5,8
1,074,A,q2,45,5,4,6
2,074,A,q3,43,3,6,8
3,074,B,q1,53,2,2,3
4,074,B,q2,46,4,3,7
5,074,B,q3,50,2,3,5
6,104,A,q1,35,10,6,9
7,104,A,q2,39,6,10,5
8,104,A,q3,38,4,10,8
9,104,B,q1,44,4,5,7


In [96]:
# sort by time
df_binary.sort_values(by=['WorkTimeInSeconds'], ascending=True, inplace=True)
df_binary

,WorkerId,WorkTimeInSeconds,subj,speech,q_id,ours_wins,ga_wins,talkg_wins,instag_wins
1145,A3VQDUUA2RI8KL,176,218,B,q3,0,1,0,0
1146,A3VQDUUA2RI8KL,176,218,B,q1,1,0,0,0
1139,A3VQDUUA2RI8KL,176,104,B,q3,0,0,0,1
1134,A3VQDUUA2RI8KL,176,074,A,q1,1,0,0,0
1140,A3VQDUUA2RI8KL,176,218,A,q1,1,0,0,0
...,...,...,...,...,...,...,...,...,...
1181,AKRPDCCP793ET,1731,304,B,q3,1,0,0,0
1180,AKRPDCCP793ET,1731,304,B,q2,1,0,0,0
1182,AKRPDCCP793ET,1731,306,A,q1,1,0,0,0
1183,AKRPDCCP793ET,1731,306,A,q2,1,0,0,0


In [107]:
df

,WorkerId,WorkTimeInSeconds,subj,speech,q_id,methodA,methodB,methodA_wins,methodB_wins,ours_wins,ga_wins,talkg_wins,instag_wins
0,A1WTHBAWHRVU4P,559,104,B,q1,ours,instag,True,False,1,0,0,0
1,A1WTHBAWHRVU4P,559,104,B,q2,ours,instag,True,False,1,0,0,0
2,A1WTHBAWHRVU4P,559,104,B,q3,ours,instag,True,False,1,0,0,0
3,A1WTHBAWHRVU4P,559,218,A,q1,ours,ga,True,False,1,0,0,0
4,A1WTHBAWHRVU4P,559,218,A,q2,ours,ga,True,False,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,A2KWD81ZH7T234,977,306,A,q2,ours,talkg,True,False,1,0,0,0
3236,A2KWD81ZH7T234,977,306,A,q3,ours,talkg,True,False,1,0,0,0
3237,A2KWD81ZH7T234,977,306,B,q1,talkg,ours,False,True,1,0,0,0
3238,A2KWD81ZH7T234,977,306,B,q2,talkg,ours,False,True,1,0,0,0


In [ ]:
# count how many times methodA has value 'ga' and methodB has value 'ga'
num_ga = df[(df['methodA'] == 'ga') | (df['methodB'] == 'ga')].shape[0]
num_ga

# 9 subj x 2 speech x 20 responses = 360 (per question)
# 9 subj x 2 speech x 20 responses x 3 questions = 1080 (total answers)

1080